In [1]:
import MySQLdb
import matplotlib
matplotlib.use ('template')
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import pylab as pl
import os
import matplotlib.colors as mcolors

In [2]:
estaciones = [91,94,93,99,140]
tipos = ['Ni','Ni','Ni','Ni','Ni']
cuencas = ['Metro_Sabaneta','Aguacatala','Puente_La33','Aula_Ambiental','Puente_Fundadores']

### Parametros estacion a actualizar

In [3]:
n = 140
ee = np.where(np.array(estaciones) == n)[0]
tip = np.array(tipos)[ee]
umb_niv = 270
path='/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/Buscador_eventos/'+np.array(cuencas)[ee][0]+'/'

FI='2017-09-10'
FF='2018-01-17'

### Find maximos

In [4]:
# Consulta en base de datos

host      ='192.168.1.74'
user      ='usrCalidad'
passw     ='aF05wnXC;'
bd_nombre ='siata'

Estaciones="SELECT Codigo,Nombreestacion, offsetN, red  FROM estaciones WHERE codigo=("+str(n)+")"
db = MySQLdb.connect(host, user,passw,bd_nombre)
db_cursor = db.cursor()
db_cursor.execute(Estaciones)
Cod = db_cursor.fetchall()
date_ini=datetime.datetime(int(FI[0:4]),int(FI[5:7]),int(FI[8:10]))
date_fin=datetime.datetime(int(FF[0:4]),int(FF[5:7]),int(FF[8:10]))
sql_datos ="SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s'), Cliente, (" +str(Cod[0][2])+"-"+tip[0]+"), calidad FROM datos WHERE cliente = "+str(n)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"

db = MySQLdb.connect(host, user,passw,bd_nombre)
db_cursor = db.cursor()
db_cursor.execute(sql_datos)
data = db_cursor.fetchall()

db_cursor.execute(Estaciones)
Nombre = db_cursor.fetchall()

In [5]:
F=[]
H=[]
ID=[]
N=[]
Calidad=[]
FH=[]

for dato in data:
    if dato[6] != 151:
        F.append(dato[1])
        H.append(dato[3])
        ID.append(dato[4])
        N.append(dato[5])
        Calidad.append(dato[6])  
    
for i in range(len(F)):
    fh=F[i]+" "+H[i]
    FH.append(fh)
    
#Hasta este punto se han extraido datos de nivel y tal cual estan en la base de datos
#vector fechas para comparar fechas de las estaciones de Nivel y Precipitacion
    
EjeX=[]
EjeY=[]
formato = '%Y-%m-%d %H:%M:%S'
C=0

ID2=[]
Calidad2=[]
FH1 = []
        
vv = np.where(np.array(Calidad) != 151)[0]
EjeY = np.array(N)[vv]
FH1 = np.array(FH)[vv]
    
for i in FH1:
    EjeX.append(datetime.datetime.strptime (i[:-3], '%Y-%m-%d %H:%M'))        

D = pd.Series(EjeY, EjeX)

#Suavizado de serie
j=0
while j<2:
    d_mva = pd.rolling_mean(D,20,min_periods=8,center=True)
    D1=d_mva
    j+=1

#Hallar maximos
Ejey = D1.values
EjeX = D1.index

ite = 3
umbral = umb_niv
busq = 30

#Hallar maximos locales

def maximos(Ejy,EjX):
    
    grad = []
    maxi = []
    picos = []
    datesM = []
    
    for j in range(len(Ejy)-1):
        if Ejy[j+1] == Ejy[j]:
            grad.append(1)
        
        if Ejy[j+1] != Ejy[j]:
            grad.append((Ejy[j+1]-Ejy[j])/abs(Ejy[j+1]-Ejy[j]))     
    
    for i in range(len(grad)-1):
        maxi.append(grad[i+1]-grad[i])
         
    pp = (np.where(np.array(maxi) == -2))[0]
    pp = pp+np.ones(len(pp))
    
    for x in range(len(pp)):
        picos.append(Ejy[int(pp[x])])
        datesM.append(EjX[int(pp[x])])

    return picos,datesM
    
for i in range(ite):
    if i == 0:
        aux = maximos(Ejey,EjeX)
    if i > 0:
        aux = maximos(aux[0],aux[1])
        
pi1 = []
da1 = []
    
for y in aux[1]:
    fa = np.where(EjeX == y)[0]
    fa = fa[len(fa)-1]
    if fa > busq:
        ma = np.where(EjeY[fa-busq:fa] == np.max(EjeY[fa-busq:fa]))[0]
        pi1.append(EjeY[fa-(busq-ma[len(ma)-1])])
        da1.append(EjeX[fa-(busq-ma[len(ma)-1])])     

dM = np.array(da1)
NM = np.array(pi1) 
dM = dM[NM >= umbral]
NM = NM[NM >= umbral]
M = pd.Series(NM, dM)

fig1=pl.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
plt.close('all')
plt.clf ()
plt.cla ()
D.plot()
D1.plot()
M.plot()
plt.savefig(path+'maximos.png')

np.savetxt(path+'fechas_1.txt',np.array(dM),fmt="%s")
np.savetxt(path+'niveles_1.txt',NM)

print '--------------------------------------------------------------------------'
print 'Graficar las hidrografas de los eventos nuevos y seleccionar los buenos'
print '--------------------------------------------------------------------------'


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=8,window=20,center=True).mean()


--------------------------------------------------------------------------
Graficar las hidrografas de los eventos nuevos y seleccionar los buenos
--------------------------------------------------------------------------


### Graficar eventos

In [6]:
fechas=np.loadtxt(path+'fechas_1.txt',dtype=str)

# open database connection
host      ='192.168.1.74'
user      ='usrCalidad'
passw     ='aF05wnXC;'
bd_nombre ='siata'

Estaciones="SELECT Codigo,Nombreestacion, offsetN, red  FROM estaciones WHERE codigo=("+str(n)+")"
db = MySQLdb.connect(host, user,passw,bd_nombre)
db_cursor = db.cursor()
db_cursor.execute(Estaciones)
Cod = db_cursor.fetchall()

In [7]:
# Itera por evento

for w in np.arange(0,len(fechas),1):
    i=fechas[w]
    print i
    dt_fecha = datetime.datetime(int(i[0][0:4]),int(i[0][5:7]),int(i[0][8:10]),int(i[1][0:2]),int(i[1][3:5]))

    #extraer de la base de datos fecha, hora y nivel el vector fecha debe quedar en el formato '%Y-%m-%d %H:%M:%S' datetime
    date_ini=dt_fecha-datetime.timedelta(minutes=60)
    date_fin=dt_fecha+datetime.timedelta(minutes=180)
    datos_n = "SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s'), (" +str(Cod[0][2])+"-"+tip[0]+"), calidad FROM datos WHERE cliente = "+str(n)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"
    db = MySQLdb.connect(host, user,passw,bd_nombre)
    db_cursor = db.cursor()
    db_cursor.execute(datos_n)
    data = db_cursor.fetchall()

    FN=[]
    N=[]
    calidad=[]

    for k in range(len(data)):
        try:
            dato = data[k]
            dt = datetime.datetime(int(dato[1][0:4]),int(dato[1][5:7]),int(dato[1][8:10]),int(dato[3][0:2]),int(dato[3][3:5]))
            FN.append(dt)
            N.append(dato[4])
            calidad.append(dato[5])
            if dt == dt_fecha:
                pp = k
        except:
            pass

    cc = np.where(np.array(calidad) != 151)[0]
    FN = np.array(FN)[cc]
    N = np.array(N)[cc]

    N=pd.Series(map(float,N),FN) # construir serie de nivel en formato pandas

    plt.close('all')
    fig=plt.figure(edgecolor='w',facecolor='w',figsize=(12,9))
    plt.plot(list(N.index),list(N.values), lw=2)
    plt.grid(linestyle='--')
    plt.xlabel("Hora")
    plt.ylabel("N (cm)")
    plt.title('Evento '+str(i[0])+' ('+str(w+1)+')')
    matplotlib.rcParams.update({'font.size': 10})

    try:
        plt.savefig(path+'Hidrografas/'+str(i[0])+'*'+str(i[1])+'.png')
    except:pass

print '-----------------------------------------'
print 'Eliminar hidrográfas para filtrar'
print '-----------------------------------------'

['2017-09-10' '01:45:00']
['2017-09-12' '23:25:00']
['2017-09-13' '18:33:00']
['2017-09-14' '19:40:00']
['2017-09-15' '05:09:00']
['2017-09-16' '04:01:00']
['2017-09-17' '02:39:00']
['2017-09-19' '17:22:00']
['2017-09-20' '09:29:00']
['2017-09-21' '15:35:00']
['2017-09-22' '00:25:00']
['2017-09-22' '16:17:00']
['2017-09-22' '20:52:00']
['2017-09-23' '18:15:00']
['2017-09-26' '15:04:00']
['2017-09-27' '08:48:00']
['2017-09-27' '15:58:00']
['2017-09-30' '05:41:00']
['2017-09-30' '18:56:00']
['2017-10-01' '15:42:00']
['2017-10-02' '17:16:00']
['2017-10-03' '13:10:00']
['2017-10-03' '16:37:00']
['2017-10-08' '16:15:00']
['2017-10-09' '15:43:00']
['2017-10-11' '06:15:00']
['2017-10-14' '23:40:00']
['2017-10-20' '22:52:00']
['2017-10-21' '17:55:00']
['2017-10-22' '15:51:00']
['2017-10-25' '16:04:00']
['2017-10-25' '19:44:00']
['2017-10-26' '12:32:00']
['2017-10-26' '15:47:00']
['2017-10-27' '17:38:00']
['2017-10-28' '13:17:00']
['2017-11-01' '01:43:00']
['2017-11-03' '02:23:00']
['2017-11-03

Eliminar las hidrografas que no se tendran en cuenta en la actualizacion y continuar con el proceso

### Unir corridas

In [8]:
fechas_toda=np.loadtxt(path+'fechas_1.txt',dtype=str)
nivel_todo=np.loadtxt(path+'niveles_1.txt')
a=os.listdir(path+'/Hidrografas/')
a.sort()
a = a[1::] 

fecha_t=[] ; hora_t=[]
for kk in fechas_toda:
    fecha_t.append(kk[0])
    hora_t.append(kk[1])

fecha=[] ; hora=[] ; nivel=[] ; fecha_str = []
for k in a:
    fecha.append(k.split('*')[0])
    hora.append(k.split('*')[1].split('.')[0])
    dd=np.where(np.array(fecha_t) == k.split('*')[0])[0]
    ee=np.where(np.array(hora_t) == k.split('*')[1].split('.')[0])[0]
    ff=list(set(dd).intersection(ee))
    nivel.append(float(nivel_todo[ff]))
    fecha_str.append(k.split('*')[0]+' '+k.split('*')[1].split('.')[0])

# Guardar nuevo 
np.savetxt(path+'fechas_1.txt',fecha_str,fmt="%s")
np.savetxt(path+'niveles_1.txt',nivel,fmt="%s")

#Actualizar archivos a unir

fechas_C1=np.loadtxt(path+'fechas.txt',dtype=str)
niveles_C1=np.loadtxt(path+'niveles.txt',dtype=str)

fechas=[]
niveles=[]

for i in range(len(fechas_C1)):

    niveles.append(niveles_C1[i])
    fechas.append(fechas_C1[i][0]+' '+fechas_C1[i][1])

for j in range(len(fecha)):

    niveles.append(nivel[j])
    fechas.append(fecha[j]+' '+hora[j])

os.system("sudo cp "+path+"/Hidrografas/*.png "+path+"/Hidrografas_todas/")

np.savetxt(path+'/niveles.txt',niveles,fmt="%s")
np.savetxt(path+'/fechas.txt',fechas,fmt="%s")

print '--------------------------------------------------------------------------------------'
print 'Revisar que los archivos niveles.txt y fechas.txt esten actualizados'
print '--------------------------------------------------------------------------------------'

--------------------------------------------------------------------------------------
Revisar que los archivos niveles.txt y fechas.txt esten actualizados
--------------------------------------------------------------------------------------


In [9]:
path

'/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/Buscador_eventos/Puente_Fundadores/'

### Elaborar graficas resumen

In [10]:
fechas=np.loadtxt(path+'fechas.txt',dtype=str)
nivp=np.genfromtxt(path+'niveles.txt',dtype=float)
dates=[]
mes=[]

for i in fechas:
    dates.append(datetime.datetime(int(i[0][0:4]),int(i[0][5:7]),int(i[0][8:10]),int(i[1][0:2]),int(i[1][3:5])))
    mes.append(int(i[0][5:7]))

In [12]:
# Histograma de niveles

clases = 6
binx = (np.max(nivp)-np.min(nivp))/clases
ejex = np.arange(np.min(nivp), np.max(nivp)+binx, binx)
hist = ((np.histogram(nivp,bins=ejex))[0])
plt.close ('all')
fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
ax1=fig.add_subplot(1,1,1)   
plt.bar(np.arange(np.min(nivp)+binx, np.max(nivp)+binx,binx)-binx/2,(hist/float(len(nivp)))*100,align='center',width=200)
plt.xlim(70,150)
ax1.set_xticks (np.arange(np.min(nivp)+binx, np.max(nivp)+binx,binx)-binx/2)
ax1.set_xticklabels (map (lambda (x) : "%.2f" % (x), np.arange(np.min(nivp)+binx, np.max(nivp)+binx,binx)-binx/2), size=12)
plt.grid()
plt.title('Numero eventos - Total = '+str(len(nivp)),size=24)
plt.xlabel('Nivel [cm]',size=24)
plt.ylabel('Frecuencia [%]',size=24)
plt.tick_params(axis='y', which='major', labelsize=22)
plt.tick_params(axis='x', which='major', labelsize=22)
plt.savefig(path+'hist_niveles.png')

# Ciclo anual

p10=np.zeros((12))
p25=np.zeros((12))
p50=np.zeros((12))
p75=np.zeros((12))
p90=np.zeros((12))


for i in range(12):
    tem = []
    for j in range(len(mes)):
        if mes[j] == i+1:
            tem.append(nivp[j])
    try:
        p10[i]=np.percentile(tem,10)
        p25[i]=np.percentile(tem,25)
        p50[i]=np.percentile(tem,50)
        p75[i]=np.percentile(tem,75)
        p90[i]=np.percentile(tem,90)
    except:
        pass

plt.close ('all')
meses=['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']
fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
ax1=fig.add_subplot(1,1,1) 
plt.plot(range(1,13,1),p10,label='Percentil 10',linewidth=2)
plt.scatter(range(1,13,1),p10,s=50,color='b')
plt.plot(range(1,13,1),p50,label='Percentil 50',linewidth=2)
plt.scatter(range(1,13,1),p50,s=50,color='g')
plt.plot(range(1,13,1),p90,label='Percentil 90',linewidth=2)
plt.scatter(range(1,13,1),p90,s=50,color='r')
plt.ylabel('Nivel [cm]',size=24)
plt.tick_params(axis='y', which='major', labelsize=22)
plt.tick_params(axis='x', which='major', labelsize=22)
plt.grid()
plt.xticks(range(1,13,1),meses,size=20)
plt.xlim(1,12)
plt.legend(loc=2)
plt.savefig(path+'ciclo_anual.png')

horas = []
for h in fechas:
    horas.append(float(h[1].split(':')[0]))

fre = np.zeros((24))
for i in horas:
    fre[int(i)] = fre[int(i)] + 1

plt.close ('all')
fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
horas=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']
plt.bar(range(1,25,1),(np.array(fre)/sum(fre))*100,width=0.9)
plt.grid()
plt.xlim(1,25)
plt.xticks(np.arange(1.5,25.5,1),horas,size=16)
plt.tick_params(axis='y', which='major', labelsize=22)
plt.ylabel('Frecuencia [%]',size=24)
plt.xlabel('Horas',size=24)
plt.savefig(path+'ciclo_horario.png')

### Copiar archivos a folder operacional

In [13]:
os.system('cp '+path+'niveles.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/niveles'+str(n)+'.txt')
os.system('cp '+path+'fechas.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/fechas'+str(n)+'.txt')

0